In [1]:
import pandas as pd 
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

In [2]:
data= pd.read_csv("D:\PFE\Fact_Table_Rev.csv", low_memory=False)

In [3]:
data

,montant_HT,montant_ttc,date_saisie,date_facture,Fk_prestation,Fk_famille_prestation,Fk_nationnalite,Fk_Societe,Fk_typesociete
0,4.0,3.571429,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
1,6.0,5.357143,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
2,8.0,7.142857,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
3,10.0,8.928571,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
4,8.5,7.589286,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
...,...,...,...,...,...,...,...,...,...
937562,10.5,9.375000,2013-09-24,2013-09-24,BOI,BAR,LI,GRINPARAPH,LAB
937563,7.0,6.250000,2013-09-24,2013-09-24,BOI,BAR,LI,GRINPARAPH,LAB
937564,3.5,3.125000,2013-09-24,2013-09-24,BOIR,CAVE,TN,STANDART,SOC
937565,196.0,175.000000,2013-09-24,2013-09-24,REP,RESTO,TN,STANDART,SOC


In [37]:
df = pd.DataFrame(data, columns=["montant_HT", "date_saisie", "Fk_famille_prestation","Fk_typesociete"])

In [38]:
df['date_saisie'] = pd.to_datetime(df['date_saisie'])

In [39]:
df

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete
0,4.0,2006-09-18,BAR,SOC
1,6.0,2006-09-18,BAR,SOC
2,8.0,2006-09-18,BAR,SOC
3,10.0,2006-09-18,BAR,SOC
4,8.5,2006-09-18,BAR,SOC
...,...,...,...,...
937562,10.5,2013-09-24,BAR,LAB
937563,7.0,2013-09-24,BAR,LAB
937564,3.5,2013-09-24,CAVE,SOC
937565,196.0,2013-09-24,RESTO,SOC


In [7]:
print(df)

        montant_HT date_saisie Fk_famille_prestation Fk_typesociete
0              4.0  2006-09-18                   BAR            SOC
1              6.0  2006-09-18                   BAR            SOC
2              8.0  2006-09-18                   BAR            SOC
3             10.0  2006-09-18                   BAR            SOC
4              8.5  2006-09-18                   BAR            SOC
...            ...         ...                   ...            ...
937562        10.5  2013-09-24                   BAR            LAB
937563         7.0  2013-09-24                   BAR            LAB
937564         3.5  2013-09-24                  CAVE            SOC
937565       196.0  2013-09-24                 RESTO            SOC
937566         3.5  2013-09-24                  CAVE            PET

[937567 rows x 4 columns]


In [40]:
df= df.drop(index=df.loc[(df['date_saisie'] < '2006-01-01') | (df['date_saisie'] > '2019-12-31')].index)

In [41]:
df= df.sort_values(by='date_saisie', ascending=True)

In [28]:
print(df)

        montant_HT date_saisie Fk_famille_prestation Fk_typesociete
0              4.0  2006-09-18                   BAR            SOC
313            0.0  2006-09-18                   BAR            SOC
314            0.0  2006-09-18                   BAR            SOC
315            0.0  2006-09-18                   BAR            SOC
316            0.0  2006-09-18                   BAR            SOC
...            ...         ...                   ...            ...
765368       120.0  2019-12-31                TORRET            SOC
765367       147.0  2019-12-31                TORRET            SOC
765366        16.0  2019-12-31                TORRET            SOC
765364        44.0  2019-12-31                TORRET            SOC
765337        12.0  2019-12-31                TORRET            SOC

[753932 rows x 4 columns]


In [42]:
label_encoder = LabelEncoder()
df['Fk_prestation_code'] = label_encoder.fit_transform(df['Fk_famille_prestation'])
label_encoder = LabelEncoder()
df['Fk_typesociete_code'] = label_encoder.fit_transform(df['Fk_typesociete'])
df.head()

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete,Fk_prestation_code,Fk_typesociete_code
0,4.0,2006-09-18,BAR,SOC,2,5
313,0.0,2006-09-18,BAR,SOC,2,5
314,0.0,2006-09-18,BAR,SOC,2,5
315,0.0,2006-09-18,BAR,SOC,2,5
316,0.0,2006-09-18,BAR,SOC,2,5


In [12]:
df

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete,Fk_prestation_code,Fk_typesociete_code
0,4.0,2006-09-18,BAR,SOC,2,5
313,0.0,2006-09-18,BAR,SOC,2,5
314,0.0,2006-09-18,BAR,SOC,2,5
315,0.0,2006-09-18,BAR,SOC,2,5
316,0.0,2006-09-18,BAR,SOC,2,5
...,...,...,...,...,...,...
765368,120.0,2019-12-31,TORRET,SOC,19,5
765367,147.0,2019-12-31,TORRET,SOC,19,5
765366,16.0,2019-12-31,TORRET,SOC,19,5
765364,44.0,2019-12-31,TORRET,SOC,19,5


In [460]:
df1 = df[['Fk_famille_prestation', 'Fk_prestation_code']].drop_duplicates()
print(df1)

       Fk_famille_prestation  Fk_prestation_code
314                      BAR                   2
324                     CAVE                   3
210                    RESTO                  15
339                   SEJOUR                  17
26497                    PTD                  14
224                       MB                   9
1719                     LIN                   6
520                      TEL                  18
2416                    DEBO                   4
33558                    P.S                  10
12757                  LOCAT                   7
96329                    DIV                   5
9208                    PISC                  12
18717                   PEIG                  11
94129                 BALNEO                   0
140920                S.COIF                  16
136211                    PR                  13
208095                   LOY                   8
416562                BANQUE                   1
456896              

In [13]:
X_train = df[(df['date_saisie'].dt.year >= 2006) & (df['date_saisie'].dt.year <= 2018)][['Fk_prestation_code']]
X_test = df[df['date_saisie'].dt.year == 2019][['Fk_prestation_code']]
y_train = df[(df['date_saisie'].dt.year >= 2006) & (df['date_saisie'].dt.year <= 2018)]['montant_HT']
y_test = df[df['date_saisie'].dt.year == 2019]['montant_HT']

Random Forest Regressor Model


In [37]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor()

In [38]:
y_pred = rf.predict(X_test)

In [39]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 135036.26844967436


In [40]:
y_pred.sum()

4468619.940700877

In [41]:
result_df = pd.DataFrame({'y_pred': y_pred, 'Fk_prestation_code': X_test['Fk_prestation_code']})
sum_by_prestation_code = result_df.groupby('Fk_prestation_code')['y_pred'].sum()
mapping_df = df[['Fk_prestation_code', 'Fk_famille_prestation']].drop_duplicates()

# Merge total_montant_HT_by_code with mapping_df on Fk_prestation_code
Predfamilleprestation= sum_by_prestation_code.reset_index().merge(mapping_df, on='Fk_prestation_code')

# Display the result
print(Predfamilleprestation)

    Fk_prestation_code        y_pred Fk_famille_prestation
0                    1  7.250650e+04                BANQUE
1                    2  1.335587e+05                   BAR
2                    3  1.624293e+05                  CAVE
3                    4  1.896823e+03                  DEBO
4                    5  3.132009e+02                   DIV
5                    6  2.849942e+04                   LIN
6                    8  2.421989e+04                   LOY
7                    9  1.493479e+04                    MB
8                   12  6.781964e+03                  PISC
9                   14  2.042801e+05                   PTD
10                  15  1.860863e+06                 RESTO
11                  17  1.299857e+06                SEJOUR
12                  18  5.242257e+02                   TEL
13                  19  6.579546e+05                TORRET


In [42]:
Predfamilleprestation.to_csv('Predfamilleprestation.csv', index=False)

SARIMAX

In [43]:
selected_columns = ['date_saisie', 'montant_HT','Fk_prestation_code']
df1 = df.loc[:, selected_columns]
df1['year'] = df['date_saisie'].dt.year.astype(int)
df1['month'] = df['date_saisie'].dt.month.astype(int)
df1['day'] = df['date_saisie'].dt.day.astype(int)
selected_columns = ['date_saisie','year','month', 'day','montant_HT','Fk_prestation_code']
df1 = df1.loc[:, selected_columns]

In [44]:
df1

,date_saisie,year,month,day,montant_HT,Fk_prestation_code
0,2006-09-18,2006,9,18,4.0,2
313,2006-09-18,2006,9,18,0.0,2
314,2006-09-18,2006,9,18,0.0,2
315,2006-09-18,2006,9,18,0.0,2
316,2006-09-18,2006,9,18,0.0,2
...,...,...,...,...,...,...
765368,2019-12-31,2019,12,31,120.0,19
765367,2019-12-31,2019,12,31,147.0,19
765366,2019-12-31,2019,12,31,16.0,19
765364,2019-12-31,2019,12,31,44.0,19


In [45]:
df1= df1.groupby([ 'date_saisie','month', 'Fk_prestation_code'])['montant_HT'].sum().reset_index()

In [47]:
df1.set_index('date_saisie', inplace=True)

In [54]:
df1

,month,Fk_prestation_code,montant_HT
date_saisie,,,
2006-09-18,9,2,111.0
2006-09-18,9,3,0.0
2006-09-19,9,2,66.0
2006-09-19,9,3,26.0
2006-09-19,9,15,171.0
...,...,...,...
2019-12-31,12,12,15.0
2019-12-31,12,14,548.0
2019-12-31,12,15,9630.5


In [50]:
index_type = type(df1.index)

# Check if it's a DateTimeIndex
is_datetime_index = isinstance(df1.index, pd.DatetimeIndex)

# Print the result
print("Index type:", index_type)
print("Is DateTimeIndex:", is_datetime_index)

Index type: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
Is DateTimeIndex: True


In [55]:
train_data = df1.loc[:'2018']
test_data = df1.loc['2019':]

In [66]:
test_data

,month,Fk_prestation_code,montant_HT
date_saisie,,,
2019-01-01,1,2,320.673
2019-01-01,1,3,10.000
2019-01-01,1,6,11.000
2019-01-01,1,9,4.673
2019-01-01,1,14,185.907
...,...,...,...
2019-12-31,12,12,15.000
2019-12-31,12,14,548.000
2019-12-31,12,15,9630.500


In [57]:
test_data['montant_HT'].sum()

4854624.2639

In [58]:
model = SARIMAX(train_data['montant_HT'],  order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
result = model.fit() 

d:\PFE\libraires\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\PFE\libraires\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\PFE\libraires\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


In [78]:
start = len(train_data)
end = len(train_data) + len(test_data) -1
predictions = result.predict(start, end).sum()
predictions

d:\PFE\libraires\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


5079910.619187778

LSTM


In [12]:
df

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete,Fk_prestation_code,Fk_typesociete_code
0,4.0,2006-09-18,BAR,SOC,2,5
313,0.0,2006-09-18,BAR,SOC,2,5
314,0.0,2006-09-18,BAR,SOC,2,5
315,0.0,2006-09-18,BAR,SOC,2,5
316,0.0,2006-09-18,BAR,SOC,2,5
...,...,...,...,...,...,...
765368,120.0,2019-12-31,TORRET,SOC,19,5
765367,147.0,2019-12-31,TORRET,SOC,19,5
765366,16.0,2019-12-31,TORRET,SOC,19,5
765364,44.0,2019-12-31,TORRET,SOC,19,5


In [176]:
selected_columns = ['date_saisie', 'montant_HT','Fk_prestation_code']
df1 = df.loc[:, selected_columns]
df1['year'] = df['date_saisie'].dt.year.astype(int)
df1['month'] = df['date_saisie'].dt.month.astype(int)
df1['day'] = df['date_saisie'].dt.day.astype(int)
selected_columns = ['year','month', 'day','montant_HT','Fk_prestation_code']
df1 = df1.loc[:, selected_columns]
df1

,year,month,day,montant_HT,Fk_prestation_code
0,2006,9,18,4.0,2
313,2006,9,18,0.0,2
314,2006,9,18,0.0,2
315,2006,9,18,0.0,2
316,2006,9,18,0.0,2
...,...,...,...,...,...
765368,2019,12,31,120.0,19
765367,2019,12,31,147.0,19
765366,2019,12,31,16.0,19
765364,2019,12,31,44.0,19


In [40]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [341]:
train_data = df1[(df1['year'] <= 2018)]
test_data = df1[df1['year'] == 2019]

In [342]:
train_data

,year,month,day,montant_HT,Fk_prestation_code
0,2006,9,18,4.0,2
313,2006,9,18,0.0,2
314,2006,9,18,0.0,2
315,2006,9,18,0.0,2
316,2006,9,18,0.0,2
...,...,...,...,...,...
729061,2018,12,31,20.0,9
729062,2018,12,31,13.0,6
729063,2018,12,31,10.0,3
727937,2018,12,31,13.0,19


In [343]:
train_data = train_data.groupby([ 'year','month','day' ,'Fk_prestation_code'])['montant_HT'].sum().reset_index()
test_data  = test_data .groupby([ 'year','month','day', 'Fk_prestation_code'])['montant_HT'].sum().reset_index()

In [344]:
train_data

,year,month,day,Fk_prestation_code,montant_HT
0,2006,9,18,2,111.000
1,2006,9,18,3,0.000
2,2006,9,19,2,66.000
3,2006,9,19,3,26.000
4,2006,9,19,15,171.000
...,...,...,...,...,...
37303,2018,12,31,14,198.037
37304,2018,12,31,15,5561.916
37305,2018,12,31,17,1023.363
37306,2018,12,31,18,8.080


In [345]:
print(test_data)

      year  month  day  Fk_prestation_code  montant_HT
0     2019      1    1                   2     320.673
1     2019      1    1                   3      10.000
2     2019      1    1                   6      11.000
3     2019      1    1                   9       4.673
4     2019      1    1                  14     185.907
...    ...    ...  ...                 ...         ...
3143  2019     12   31                  12      15.000
3144  2019     12   31                  14     548.000
3145  2019     12   31                  15    9630.500
3146  2019     12   31                  17    2790.400
3147  2019     12   31                  19    2898.000

[3148 rows x 5 columns]


In [346]:
train_data_array = train_data.values
test_data_array = test_data.values

In [347]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train_data = scaler.fit_transform(train_data_array)
scaled_test_data = scaler.transform(test_data_array) 

In [348]:
seq_length = 12  
X_train, y_train = create_sequences(scaled_train_data, seq_length)
X_test, y_test = create_sequences(scaled_test_data, seq_length)

In [349]:
from keras.layers import Input, LSTM, Dense
from keras.models import Model

input_shape = (X_train.shape[1], X_train.shape[2])

input_layer = Input(shape=input_shape)
lstm_layer = LSTM(units=50)(input_layer)
output_layer = Dense(units=1)(lstm_layer)

model = Model(inputs=input_layer, outputs=output_layer)


In [350]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [351]:
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

Epoch 1/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0016
Epoch 2/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0011
Epoch 3/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0011
Epoch 4/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0010
Epoch 5/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 9.8984e-04
Epoch 6/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 9.8589e-04
Epoch 7/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 9.4377e-04
Epoch 8/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 8.7510e-04
Epoch 9/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0011
Epoch 10/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 9.4106e-04
Epoch 11/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0010
Epoch 12/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 8.7553e-04
Epoch 13/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 8.2318e-04
Epoch 14/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - 

In [352]:
predicted_data = model.predict(X_test)

98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [353]:
predicted_data.sum()

205.41223

In [357]:
# Combine X_test features with y_test data
combined_y_test_data = np.hstack((X_test[:, -1], predicted_data.reshape(-1, 1)))

# Inverse transform to get real values of y_test
predicted_data_real = scaler.inverse_transform(combined_y_test_data)

# Sum of real values of y_test
predicted_data_sum = np.sum(predicted_data_real[:, -1])
print("Sum  of predicted_data:", predicted_data_sum)

Sum  of predicted_data: 4884440.051113475
